In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

## Using a loop to retrieve data from past seasons

In [2]:
standing_url = "https://fbref.com/en/comps/12/La-Liga-Stats"

In [3]:
years = list(range(2023, 2022, -1))
years = list([2023])

all_matches = []

In [7]:
for year in years: 
    data = requests.get(standing_url)
    soup = BeautifulSoup(data.text)
    standing_table = soup.select("table.stats_table")[0]

    links = standing_table.find_all("a")
    links = [l.get("href") for l in links]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"http://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standing_url = f"https://fbref.com/{previous_season}"


    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", "")

        data = requests.get(team_url)
        matches = pd.read_html(data.text, match = "Scores & Fixtures")[0]

        soup = BeautifulSoup(data.text)
        links = soup.find_all("a")
        links = [l.get("href") for l in links]
        links = [l for l in links if l and "all_comps/shooting/" in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match = "Shooting")[0]
        shooting.columns = shooting.columns.droplevel()

        try:
            team_data = pd.merge(left = matches, right = shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on = "Date")
        except ValueError:
            continue

        #team_data = team_data[team_data["Comp"] == "La Liga"]
        team_data["Season"] = year
        team_data["Team"] = team_name

        all_matches.append(team_data)
        time.sleep(10)

IndexError: list index out of range

In [5]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]
match_df.to_csv("LaLiga_Matches")
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team


## work around for the limited rate of the page

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Scraper API endpoint
api_url = "https://api.scraperapi.com"

# Scraper API key
api_key = ""

standing_url = "https://fbref.com/en/comps/12/La-Liga-Stats"
years = [2023]
all_matches = []

for year in years:
    # Construct the URL with the Scraper API parameters
    params = {
        "api_key": api_key,
        "url": standing_url
    }
    data = requests.get(api_url, params=params)

    soup = BeautifulSoup(data.text, "html.parser")
    standing_table = soup.select("table.stats_table")[0]

    links = standing_table.find_all("a")
    links = [l.get("href") for l in links]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"http://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standing_url = f"https://fbref.com/{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", "")

        # Make the request to Scraper API for team_url
        params["url"] = team_url
        data = requests.get(api_url, params=params)

        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

        soup = BeautifulSoup(data.text, "html.parser")
        links = soup.find_all("a")
        links = [l.get("href") for l in links]
        links = [l for l in links if l and "all_comps/shooting/" in l]

        # Make the request to Scraper API for shooting stats
        params["url"] = f"https://fbref.com{links[0]}"
        data = requests.get(api_url, params=params)

        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()

        try:
            team_data = pd.merge(left=matches, right=shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue

        #team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["Team"] = team_name

        all_matches.append(team_data)
        time.sleep(10)

match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]
match_df.to_csv("LaLiga_Matches")
match_df

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))